In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

'''---------------= INPUT DATA ----------------'''

'''----- 배당은 각각 한국전력 배당수익률 5년 평균치 이용, 한국전력 = 5.27% / 하나금융지주 = 4.82% ----'''

num = 10000
num_asset = 2
step = 32
T = 2
dt = T / step #/num_time_step
r = 0.03 
q1 = 0.0527
q2 = 0.0482
rho = 0.2
sigma1 = 0.3 
sigma2 = 0.4 
rate = 0.125

s1_0 = 100
s2_0 = 100

case_dict = {
    1 : s1_0 * (1 + 0.5 * rate) / (1 + 0.5 * r),
    2 : s1_0 * (1 + 1.0 * rate) / (1 + 1.0 * r),
    3 : s1_0 * (1 + 1.5 * rate) / (1 + 1.5 * r),
    4 : s1_0 * (1 + 2.0 * rate) / (1 + 2.0 * r),
    5 : s1_0 / (1 + 2.0 * r),
    6 : s1_0 / (1 + 2.0 * r)
}

'''---------------- Equations and Simulation -----------------'''

path1 = np.ones((num, 1)) * s1_0
path2 = np.ones((num, 1)) * s2_0

z1 = np.random.normal(size=(step, num)).T
z2 = rho * z1 + np.sqrt(1-rho**2) * np.random.normal(size=(step, num)).T

x1 = np.exp( (r - q1 - 0.5 * (sigma1 **2)) * dt + sigma1 * np.sqrt(dt) * z1)
x2 = np.exp( (r - q2 - 0.5 * (sigma1 **2)) * dt + sigma1 * np.sqrt(dt) * z2)

for i in range(step):
    next_path = x1[:,i].reshape(num, 1) * path1[:, -1].reshape(num, 1)
    path1 = np.c_[path1, next_path]

for i in range(step):
    next_path = x2[:,i].reshape(num, 1) * path2[:, -1].reshape(num, 1)
    path2 = np.c_[path2, next_path]

path1_df = pd.DataFrame(data=path1, index=[x for x in range(num)], columns=[x for x in range(step+1)])
path2_df = pd.DataFrame(data=path2, index=[x for x in range(num)], columns=[x for x in range(step+1)])

check1 = pd.DataFrame(data=path1, index=[[x for x in range(num)],[0 for x in range(num)]], columns=[x for x in range(step+1)])
check2 = pd.DataFrame(data=path2, index=[[x for x in range(num)],[1 for x in range(num)]], columns=[x for x in range(step+1)])


df = pd.concat([check1, check2]).sort_index()
display(df)


'''----------------------- Case by Case --------------------------'''

'''Case1 Check'''
'''모두 6개월 시점에서 85 이상일때'''
case1_arr = (((path1_df[8] >= 85) * 1) * ((path2_df[8] >= 85) * 1))

'''Case2 Check'''
'''모두 12개월 시점에서 80 이상일때'''
case2_arr = (((path1_df[16] >= 80) * 1) * ((path2_df[16] >= 80) * 1))
case2_arr = np.where((case2_arr - case1_arr) > 0, 1, 0)

'''Case3 Check'''
'''모두 16개월 시점에서 75 이상일때'''
case3_arr = (((path1_df[24] >= 75) * 1) * ((path2_df[24] >= 75) * 1))
case3_arr = np.where((case3_arr - case2_arr - case1_arr) > 0, 1, 0)

'''Case1, Case2, Case3 조기상환'''
early_redemption_arr = (((((case1_arr - 1) ** 2) * ((case2_arr - 1) ** 2) * ((case3_arr - 1) ** 2)) - 1) ** 2)

'''Case4 Check'''
'''모두 24개월 시점에서 70 이상일때'''
case4_arr = (((path1_df[32] >= 70) * 1) * ((path2_df[32] >= 70) * 1))
case4_arr = np.where(case4_arr - case3_arr - case2_arr - case1_arr > 0, 1 ,0)

'''Case5 Check, 조기상환 경우는 제외함'''
'''둘다 한번도 60 미만으로 하락한 적이 없고, 만기 평가 가격이 한 종목 이라도 70 미만인 경우'''
case5_arr = ( (((path1_df >= 60).prod(axis=1) * (path2_df >= 60).prod(axis=1))) * (((((path1_df[32] >= 70) * 1) * ((path2_df[32] >= 70) * 1)) - 1) ** 2) )
case5_arr = np.where((case5_arr - early_redemption_arr) > 0, 1, 0)

'''Case6 Check, 조기상환 경우는 제외함'''
'''둘다 한번이라도 60 미만으로 하락한 적이 있고, 만기 평가 가격이 한 종목 이라도 70 미만인 경우'''
case6_arr = ((((path1_df >= 60).prod(axis=1) * (path2_df >= 60).prod(axis=1)) - 1 ) **2) * (((((path1_df[32] >= 70) * 1) * ((path2_df[32] >= 70))) - 1) **2)
case6_arr = np.where((case6_arr - early_redemption_arr) > 0, 1, 0)

case = case1_arr * 1 + case2_arr * 2 + case3_arr * 3 + case4_arr * 4 + case5_arr * 5 + case6_arr * 6

'''--------------------- Values from case by case -------------------------'''

path1_df['case'] = case
path2_df['case'] = case


y_arr = np.where(path1_df['case'] == 1, case_dict[1],\
        np.where(path1_df['case'] == 2, case_dict[2],\
        np.where(path1_df['case'] == 3, case_dict[3],\
        np.where(path1_df['case'] == 4, case_dict[4],\
        np.where(path1_df['case'] == 5, case_dict[5],\
        np.where(path1_df['case'] == 6, 0, 0))))))

y_arr = y_arr + np.c_[((path1_df['case'] == 6) * path1_df[step])/s1_0, ((path1_df['case'] == 6) * path2_df[step])/s2_0].min(axis=1) * case_dict[6]

path1_df['Y'] = y_arr
path2_df['Y'] = y_arr

display(path1_df[['case', 'Y']])


'''------------------------- Price and Standard, and DF sorting -------------------------'''

value = path2_df['Y'].mean()
sample_std = np.sqrt(sum((path2_df['Y'] - value)**2)/(num-1))
mc_simul_error = sample_std/ np.sqrt(num)

prob_df = pd.DataFrame(data=path1_df.groupby('case').size().index, columns=['case']).set_index('case')
prob_df['count'] = path1_df.groupby('case').size()
prob_df['prob(%)'] = (path1_df.groupby('case').size() / num) * 100

display(prob_df)

print(f'ELS price = {value}, standard error = {mc_simul_error}')

0           1           2           3           4           5   \
0    0  100.0  111.996456  103.764325  101.630867   92.951174   84.655541   
     1  100.0  106.978370  102.972622  100.467584   91.772430   92.160469   
1    0  100.0  109.193943  109.289876  106.943367  101.379163  111.553400   
     1  100.0   98.440849   99.771361  104.373029   83.149381   85.921133   
2    0  100.0  106.704772  110.949131  122.219861  125.044943  127.307886   
...       ...         ...         ...         ...         ...         ...   
9997 1  100.0  101.197321  111.225433  114.751107  125.915974  115.159639   
9998 0  100.0   90.393958   81.632312   87.798146   86.770425   88.180952   
     1  100.0  103.086217  103.162887  101.404445  102.410969   94.582181   
9999 0  100.0  110.625068   92.093721   98.686009   94.097229   88.873275   
     1  100.0   95.963073  105.790241  109.834451  106.650526  100.584004   

                6           7           8           9   ...          23  \
0    0   75.714746   80.765671   79.919415   77.547947  ...   62.044603   
     1   82.584893   82.166173   76.190536   69.440396  ...   46.232469   
1    0  109.113814  102.818524  100.931026  113.740067  ...   93.184304   
     1   91.328483   86.015097  109.829102  111.353991  ...   60.467961   
2    0  132.073800  132.234722  135.675686  132.771262  ...  143.156895   
...            ...         ...         ...         ...  ...         ...   
9997 1  101.348007   95.773051   80.463954   68.892041  ...   78.668089   
9998 0   94.628894  101.921032  102.876280   94.948318  ...   94.822553   
     1   87.122948   94.478833   89.780634   90.550617  ...  110.720695   
9999 0   98.487869  112.115487  101.876566  106.070776  ...   81.091612   
     1  111.704321  107.074314  102.510224  127.792094  ...  108.165282   

                24          25          26          27          28  \
0    0   68.571636   72.802637   68.140527   73.913329   77.466209   
     1   46.588943   47.300330   41.261255   39.779049   35.484678   
1    0   99.470978  102.543433   96.436269   96.461631   96.136629   
     1   60.906351   62.154699   59.700386   67.659941   58.868611   
2    0  131.165429  130.902623  130.013771  124.753474  126.308077   
...            ...         ...         ...         ...         ...   
9997 1   77.075595   76.453540   77.486286   74.184897   67.892274   
9998 0   96.055323  100.395198  102.619747   99.980017  111.471478   
     1  119.101581  111.496202  106.691589  108.830204  118.407080   
9999 0   83.141268   81.832264   77.286845   73.763827   75.644804   
     1   99.541208  105.844328   95.431002   89.053739   96.192509   

                29          30          31          32  
0    0   72.667695   74.146798   70.614986   64.564631  
     1   35.066709   39.224753   40.624352   39.435159  
1    0   95.086103   89.601925   79.639991   67.526608  
     1   55.092406   55.402406   53.144538   50.924324  
2    0  115.440073  109.861290  106.622642   94.783523  
...            ...         ...         ...         ...  
9997 1   72.614581   66.848750   68.035228   71.295375  
9998 0  108.917035   93.655892   87.957597   93.960018  
     1  128.629663  116.728560  103.772570  100.743420  
9999 0   89.257259   92.500564   88.388192   94.443513  
     1  101.307529  102.619851  118.901357  123.830043  

[20000 rows x 33 columns]

,case,Y
0,6,37.202980
1,1,104.679803
2,1,104.679803
3,1,104.679803
4,1,104.679803
...,...,...
9995,1,104.679803
9996,4,117.924528
9997,3,113.636364
9998,1,104.679803


,count,prob(%)
case,,
1,5530,55.30
2,1193,11.93
3,595,5.95
4,396,3.96
5,87,0.87
6,2199,21.99


ELS price = 93.31414556157333, standard error = 0.25737438918070166
